In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import ipywidgets as widgets
from IPython.display import display
from collections import defaultdict
import matplotlib.lines as mlines
from scipy.spatial import cKDTree


# ========== 1) Read Data ==========
df = pd.read_excel("updated_data_01.xlsx")

# Creating the new 'Context' value using the floor of the average of Referential_Context and Environmental_Context
df['Context'] = ((df['Referential_Context'] + df['Environmental_Context']) / 2).apply(lambda x: int(x))

# Mapping from cluster numbers to names
# Reordered as requested: Contextual, Hypernatural, Embedded, Superimposed, Situated
cluster_names = {
    1: "Superimposed",
    2: "Situated",
    3: "Hypernatural",
    4: "Embedded", 
    5: "Contextual"
}

# Reordered cluster list for display order
ordered_clusters = [5, 3, 4, 1, 2]  # Contextual, Hypernatural, Embedded, Superimposed, Situated

# ========== 2) Group Points by Cluster ==========
cluster_data = defaultdict(list)
all_points = []  # Store all points for sorting by diegetic values

for _, row in df.iterrows():
    # Skip any points with cluster 6
    if row['cluster'] == 6:
        continue
        
    point_info = {
        'label':   str(row['idx']),
        'idx':     row['idx'],  # Store original idx for tie-breaking
        'x':       row['Spatial_Proximity'],
        'y':       row['Context'],
        'z':       row['Coherence'],  # Z-axis
        'size':    row['Concreteness'],
        'cluster': row['cluster'],    # 1..5
        'type':    row['Type'],       # SG or DV
        'diegetic_value': row['Diegetic Values']  # Add diegetic value for sorting
    }
    cluster_data[row['cluster']].append(point_info)
    all_points.append(point_info)  # Add to flattened list

# Sort all points by diegetic value (high to low), breaking ties with idx
sorted_points = sorted(all_points, key=lambda p: (-p['diegetic_value'], p['idx']))

# Extract labels in sorted order
all_labels = [point['label'] for point in sorted_points]

# Create a mapping from node label to cluster for button coloring
label_to_cluster = {}
for c_id, points in cluster_data.items():
    for point in points:
        label_to_cluster[point['label']] = c_id

# ========== 3) Prepare Widgets ==========

# --- A) Large Image widget at bottom ---
image_widget = widgets.Image(format='png')
image_widget.layout.width = "1280px"
image_widget.layout.height = "720px"

try:
    with open('images/Slide1.PNG', 'rb') as f:
        image_widget.value = f.read()
except FileNotFoundError:
    print("Warning: 'Slide1.PNG' not found!")

# --- B) 3D plot output widget ---
plot_output = widgets.Output()
# Set larger width for plot output
plot_output.layout.width = '1000px'
plot_output.layout.min_width = '700px'
# *** ADJUST THIS VALUE TO SHIFT THE PLOT RIGHT ***
plot_output.layout.margin = '0 0 0 70px'  # Format: 'top right bottom left' - increase left margin to shift right

# --- C) Sliders (Azimuth & Elevation) at top ---
azimuth_slider = widgets.IntSlider(min=0,  max=360, step=5, value=135, description='Azimuth')
elevation_slider = widgets.IntSlider(min=0, max=90,  step=5, value=25,  description='Elevation')

def refresh_plot(change):
    update_plot(azimuth_slider.value, elevation_slider.value)

for slider in (azimuth_slider, elevation_slider):
    slider.observe(refresh_plot, names='value')

# ========== 4) Colors, Filter, and Display Options ==========
cluster_colors = [
    "#FF9FCF",  # Cluster 1 - Superimposed
    "#8FE38F",  # Cluster 2 - Situated
    "#8CCBFF",  # Cluster 3 - Hypernatural
    "#FAFA7E",  # Cluster 4 - Embedded
    "#FF8E5B",  # Cluster 5 - Contextual
]
cluster_filter = None  # Which cluster is highlighted?
type_filter = None     # Type filter - 'SG' or 'DV'
show_labels = True     # Toggle for label visibility

# ========== 5) Smart Label Placement Function ==========
def calculate_label_positions(all_cluster_points):
    """
    Calculate optimal label positions to avoid overlaps across all clusters.
    Returns a dictionary mapping (cluster_id, point_label) to (x, y, z) position.
    """
    # Extract all points across clusters for density calculation
    all_points = []
    point_metadata = []  # Store cluster_id and label for each point

    for c_id, points in all_cluster_points.items():
        for p in points:
            all_points.append([p['x'], p['y'], p['z']])
            point_metadata.append((c_id, p['label']))

    # Use KDTree to calculate point density
    if not all_points:
        return {}  # No points to process

    kdtree = cKDTree(all_points)

    # Calculate optimal label positions
    label_positions = {}

    for i, (point_coords, meta) in enumerate(zip(all_points, point_metadata)):
        c_id, label = meta
        x, y, z = point_coords

        # Find number of neighbors within radius to determine density
        neighbors = kdtree.query_ball_point(point_coords, r=0.5)
        density = len(neighbors)

        # Deterministic seed for consistent offsets
        seed = int(label) if label.isdigit() else hash(label)
        np.random.seed(seed)

        if density > 3:  # High density area - use offset with leader line
            # Choose offset direction based on point position and add some randomness
            base_offset = 0.4
            random_factor = 0.2

            # Compute offset direction away from the center
            center = np.array([3, 3, 3])  # Assuming plot center is (3,3,3)
            point = np.array([x, y, z])
            direction = point - center

            # Normalize direction vector (but handle zero case)
            norm = np.linalg.norm(direction)
            if norm > 0.001:
                direction = direction / norm
            else:
                direction = np.array([1, 0, 0])  # Default direction if at center

            # Add random perturbation to direction
            random_perturb = np.random.uniform(-random_factor, random_factor, 3)
            direction = direction + random_perturb

            # Compute final offset
            dx, dy, dz = direction * base_offset

            # Ensure z offset is positive (label above point)
            dz = abs(dz) + 0.1

            label_positions[(c_id, label)] = {
                'type': 'leader_line',
                'node_pos': (x, y, z),
                'label_pos': (x + dx, y + dy, z + dz)
            }
        else:
            # Just place label with small offset above
            z_offset = 0.15 + np.random.uniform(0, 0.05)
            x_offset = np.random.uniform(-0.05, 0.05)
            y_offset = np.random.uniform(-0.05, 0.05)

            label_positions[(c_id, label)] = {
                'type': 'direct',
                'label_pos': (x + x_offset, y + y_offset, z + z_offset)
            }

    return label_positions

# ========== 6) 3D Plot Function ==========
def update_plot(azimuth=135, elevation=25):
    """Draw a 3D plot with smart label placement to avoid overlaps."""
    with plot_output:
        plot_output.clear_output(wait=True)

        fig = plt.figure(figsize=(20, 12))  # Increased from (18, 10)
        ax = fig.add_subplot(111, projection='3d')

        # Axes & camera
        ax.set_xlim(1, 5)
        ax.set_ylim(1, 5)
        ax.set_zlim(1, 5)
        ax.view_init(elev=elevation, azim=azimuth)

        # Floor polygon
        floor_vertices = [[1,1,1], [5,1,1], [5,5,1], [1,5,1]]
        floor_polygon = Poly3DCollection([floor_vertices], color='#C0C0C0', alpha=0.2)
        ax.add_collection3d(floor_polygon)

        # Calculate optimal label positions for all points
        label_positions = calculate_label_positions(cluster_data)

        # Plot each cluster in turn
        for c_id in range(1, 6):  # Changed from 1-6 to 1-5
            points = cluster_data[c_id]
            if not points:
                continue

            for point in points:
                # Skip if not matching type filter (if active)
                if type_filter is not None and point['type'] != type_filter:
                    continue

                x, y, z = point['x'], point['y'], point['z']

                # Node size from Concreteness
                node_size = 150 + (point['size'] - 1) * 80

                # Highlight or dim based on cluster and type filters
                is_highlighted = (
                    (cluster_filter is None or c_id == cluster_filter) and
                    (type_filter is None or point['type'] == type_filter)
                )
                alpha_val = 1.0 if is_highlighted else 0.05

                color_for_cluster = cluster_colors[c_id - 1]
                ax.scatter(
                    x, y, z,
                    c=color_for_cluster,
                    s=node_size,
                    alpha=alpha_val,
                    edgecolor='black'
                )

                # Add label if appropriate
                if show_labels:
                    label = point['label']

                    # Get the label positioning info
                    pos_info = label_positions.get((c_id, label))
                    if not pos_info:
                        continue  # Skip if position info not found

                    if pos_info['type'] == 'leader_line':
                        # Draw a leader line to connect point and label
                        node_pos = pos_info['node_pos']
                        label_pos = pos_info['label_pos']

                        # Draw leader line
                        ax.plot(
                            [node_pos[0], label_pos[0]],
                            [node_pos[1], label_pos[1]],
                            [node_pos[2], label_pos[2]],
                            color='gray',
                            linestyle='-',
                            linewidth=0.7,
                            alpha=alpha_val
                        )

                        # Add label at end of leader line with colored border
                        ax.text(
                            label_pos[0], label_pos[1], label_pos[2],
                            label,
                            fontsize=8 + (point['size'] * 0.2),
                            ha='center',
                            va='center',
                            color='black',
                            alpha=alpha_val,
                            zorder=20,
                            bbox=dict(
                                boxstyle="round,pad=0.2",
                                fc="white",
                                ec=color_for_cluster,  # Use cluster color for border
                                lw=1.5,                # Make border slightly thicker
                                alpha=0.8 * alpha_val
                            )
                        )
                    else:  # Direct label
                        # Place label directly above point with colored border
                        label_pos = pos_info['label_pos']
                        ax.text(
                            label_pos[0], label_pos[1], label_pos[2],
                            label,
                            fontsize=8 + (point['size'] * 0.2),
                            ha='center',
                            va='center',
                            color='black',
                            alpha=alpha_val,
                            zorder=20,
                            bbox=dict(
                                boxstyle="round,pad=0.2",
                                fc="white",
                                ec=color_for_cluster,  # Use cluster color for border
                                lw=1,                 # Border thickness
                                alpha=0.7 * alpha_val
                            )
                        )
                else:
                    # When labels are hidden, still draw small annotation dots at the nodes
                    # to indicate that there are nodes with labels at these positions
                    ax.scatter(
                        x, y, z + 0.1,  # Slight offset above the main node
                        c='white',
                        s=20,  # Small dot
                        alpha=alpha_val * 0.5,
                        edgecolor=color_for_cluster,
                        linewidths=1
                    )

        # Optional embedded legend
        legend_elems = []
        
        # Use the ordered_clusters list to ensure the desired order in the legend
        for c_id in ordered_clusters:
            leg = mlines.Line2D(
                [0],[0],
                marker='o',
                color='w',
                label=cluster_names[c_id],  # Use cluster name
                markerfacecolor=cluster_colors[c_id-1],
                markersize=9
            )
            legend_elems.append(leg)

        # Add type legend items if needed
        if type_filter is not None:
            type_legend = mlines.Line2D(
                [0],[0],
                marker='s',
                color='w',
                markerfacecolor='gray',
                label=f"Type: {'Spatial Guidance' if type_filter == 'SG' else 'Data Visualization'}",
                markersize=9
            )
            legend_elems.append(type_legend)

        ax.legend(
            handles=legend_elems,
            title="Clusters",
            loc='upper left',
            bbox_to_anchor=(1.08, 1)
        )

        # Update title
        title_parts = []

        if cluster_filter is not None:
            # Count points in current cluster that match type filter (if active)
            filtered_nodes = [p['label'] for p in cluster_data[cluster_filter]
                             if type_filter is None or p['type'] == type_filter]

            if filtered_nodes:
                nodes_str = ', '.join(sorted(filtered_nodes, key=lambda x: int(x) if x.isdigit() else float('inf')))
                title_parts.append(f"{cluster_names[cluster_filter]} Nodes: {nodes_str}")  # Use cluster name
            else:
                title_parts.append(f"{cluster_names[cluster_filter]} (No matching nodes)")  # Use cluster name

        if type_filter is not None:
            type_name = "Spatial Guidance" if type_filter == "SG" else "Data Visualization"
            title_parts.append(f"Type: {type_name}")

        if title_parts:
            title = "3D Plot - " + " / ".join(title_parts)
        else:
            title = "3D Plot with Smart Label Placement"

        ax.set_xlabel('Spatial Proximity (X)', fontsize=12)
        ax.set_ylabel('Environmental Context (Y)', fontsize=12)
        ax.set_zlabel('Coherence (Z)', fontsize=12, labelpad=10)
        ax.set_title(title, fontsize=14, pad=20)

        plt.show()

# Initial plot
update_plot(azimuth_slider.value, elevation_slider.value)

# ========== 7) Toggle Labels and Cluster Buttons ==========
def toggle_labels(btn):
    global show_labels
    show_labels = not show_labels
    btn.description = "Show Labels" if not show_labels else "Hide Labels"
    update_plot(azimuth_slider.value, elevation_slider.value)

toggle_labels_btn = widgets.Button(
    description="Hide Labels",
    layout=widgets.Layout(width='100px', height='30px'),  # Reduced from 120px width, 35px height
    button_style='warning'
)
toggle_labels_btn.on_click(toggle_labels)

def on_cluster_button_click(btn):
    global cluster_filter
    # Get cluster id from button mapping
    for c_id, c_name in cluster_names.items():
        if btn.description == c_name:
            cluster_filter = c_id
            break
    update_plot(azimuth_slider.value, elevation_slider.value)

# Create cluster buttons in the specified order
cluster_buttons = []
for c_id in ordered_clusters:  # Use the ordered list
    b = widgets.Button(
        description=cluster_names[c_id],
        layout=widgets.Layout(width='100px', height='30px')  # Reduced from 120px width, 35px height
    )
    b.style.button_color = cluster_colors[c_id - 1]
    b.on_click(on_cluster_button_click)
    cluster_buttons.append(b)

def on_show_all_click(btn):
    global cluster_filter, type_filter
    cluster_filter = None
    type_filter = None
    update_plot(azimuth_slider.value, elevation_slider.value)

show_all_btn = widgets.Button(
    description="Show All", 
    layout=widgets.Layout(width='100px', height='30px')  # Reduced from 120px width, 35px height
)
show_all_btn.on_click(on_show_all_click)

# ========== Type Filter Buttons ==========
def on_type_button_click(btn):
    global type_filter
    if btn.description == "Spatial Guidance":
        type_filter = "SG"
    else:  # Data Visualization
        type_filter = "DV"
    update_plot(azimuth_slider.value, elevation_slider.value)

# Create the two type filter buttons with smaller size
sg_button = widgets.Button(
    description="Spatial Guidance",
    layout=widgets.Layout(width='130px', height='30px'),  # Reduced from 120px width, 35px height
    button_style='info'
)
sg_button.on_click(on_type_button_click)

dv_button = widgets.Button(
    description="Data Visualization",
    layout=widgets.Layout(width='130px', height='30px'),  # Reduced from 120px width, 35px height
    button_style='info'
)
dv_button.on_click(on_type_button_click)

# ========== 8) "Scale" Placeholder Label ==========
scale_label = widgets.Label(value="Clusters Legend")

# ========== 9) Node Buttons (6 columns × 10 rows) ==========
def on_node_button_click(node_label):
    print(f"Node {node_label} selected.")
    image_path = f"images/Slide{node_label}.PNG"
    try:
        with open(image_path, 'rb') as f:
            image_widget.value = f.read()
    except FileNotFoundError:
        print(f"Warning: '{image_path}' not found. Showing default image.")
        try:
            with open('images/Slide1.PNG', 'rb') as ff:
                image_widget.value = ff.read()
        except FileNotFoundError:
            pass

# Create one button per label with cluster coloring but smaller size
node_buttons = []
for lbl in all_labels:
    btn = widgets.Button(
        description=lbl,
        layout=widgets.Layout(width='35px', height='25px')  # Reduced from 40px width, 30px height
    )

    # Apply cluster color to the button based on the label's cluster
    if lbl in label_to_cluster:
        cluster_id = label_to_cluster[lbl]
        cluster_color = cluster_colors[cluster_id - 1]
        btn.style.button_color = cluster_color

    btn.on_click(lambda b, lab=lbl: on_node_button_click(lab))
    node_buttons.append(btn)

# Arrange node buttons in rows and columns
button_rows = []
for i in range(0, len(node_buttons), 6):
    row_btns = node_buttons[i:i+6]
    # Add small padding between buttons
    row_box = widgets.HBox(row_btns, layout=widgets.Layout(
        justify_content='center', 
        padding='1px',  # Reduced from 2px
        margin='1px'    # Reduced from 2px
    ))
    button_rows.append(row_box)

buttons_box = widgets.VBox(button_rows)

# Create a container for buttons box with reduced width
buttons_box_container = widgets.Box(
    [buttons_box],
    layout=widgets.Layout(width='250px', min_width='250px')  # Reduced from 300px
)

# ========== 10) Final Layout with Modified Alignment ==========

# Row 0: Sliders (Azimuth + Elevation), horizontally centered
sliders_box = widgets.HBox(
    [azimuth_slider, elevation_slider],
    layout=widgets.Layout(
        justify_content='center', 
        width='100%',
        margin='10px 0 20px 0'
    )
)

# Add the two type buttons vertically under the cluster buttons
all_filter_buttons = [toggle_labels_btn, show_all_btn] + cluster_buttons + [sg_button, dv_button]

# Improved spacing between buttons
filter_buttons_vbox = widgets.VBox(
    all_filter_buttons, 
    layout=widgets.Layout(
        align_items='center',
        justify_content='space-between',
        margin='3px 0',  # Reduced from 5px
        padding='3px'    # Reduced from 5px
    )
)

# Make sure the left column has a fixed width and doesn't collapse
left_column = widgets.VBox(
    [scale_label, filter_buttons_vbox],
    layout=widgets.Layout(
        width='120px',   # Reduced from 150px
        min_width='120px', # Reduced from 150px
        align_items='center',
        margin='0 5px 0 0'  # Reduced right margin from 10px to 5px
    )
)

# Layout with the right box giving extra space for plot to shift right
middle_row = widgets.HBox(
    [left_column, plot_output, buttons_box_container],
    layout=widgets.Layout(
        width='100%', 
        justify_content='flex-start',  # Changed from center to flex-start
        align_items='center',
        margin='0 0 20px 0'
    )
)

# Bottom row: large image, centered
bottom_row = widgets.HBox(
    [image_widget],
    layout=widgets.Layout(
        justify_content='center', 
        width='100%',
        margin='10px 0'
    )
)

# Ensure the final layout has proper spacing
final_layout = widgets.VBox(
    [sliders_box, middle_row, bottom_row],
    layout=widgets.Layout(
        width='100%', 
        padding='10px',  # Reduced from 15px
        margin='10px'
    )
)

# Display the final layout
display(final_layout)

# Add CSS to help with Voila rendering
from IPython.display import HTML
display(HTML("""
<style>
.jupyter-widgets {
    min-height: 0;
    height: auto;
}
.widget-button {
    margin: 2px;  /* Reduced from 3px */
}
</style>
"""))